In [1]:
# imports
import pandas as pd
import numpy as np
import random
import statistics as stat

traindf = pd.read_pickle("./pickles/df_small_clean.pkl")

# display(traindf[traindf["position"] == 11])

In [2]:
def random_solution(df):
    
    # csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    
    ids =  pd.unique(column_values) 
    
    all_ranks = []
    
    for i, ideetje in enumerate(ids):
        df_temp = df[df["srch_id"] == ideetje]
        
        ranks = df_temp["position"].values
        
        # shuffle ranks
        random.shuffle(ranks)
        all_ranks.extend(ranks)

    df["predicted_rank"] = all_ranks
    return df

In [3]:
def indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked):
    # holder for ideal score
    idcg = 0
    
    # holder for our score
    dcg = 0

    # itereer over elk resultaat in de search
    for i in range(len(true_rank)):
        # hotels die geboekt zijn zijn heel belangrijk (5)
        if booked[i] == 1:
            
            # increment ideal score
            if i != 0:
                idcg = idcg + (5 / np.log2(i+1))
            else:
                idcg += 5

            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                if i != 0:
                    dcg = dcg + (5 / np.log2(i+1))
                else:
                    dcg += 5
                
        # hotels waarop geklikt is zijn een beetje belangrijk (1)
        elif clicked[i] == 1:

            # increment ideal score
            if i != 0:
                idcg = idcg + (1 / np.log2(i+1))
            else:
                idcg += 1
            
            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                dcg = dcg + (1 / np.log2(i+2))

    if idcg == 0:
        return dcg
    return dcg/idcg

In [4]:
# score function for every search (thus iterate over groupby object)
def dataframe_ndcg_score(solution):
    all_scores = []
    for srchid, group in solution.groupby('srch_id'):

        # true_rank
        true_rank = np.asarray(group['position'])

        # predicted_rank
        predicted_rank = np.asarray(group['predicted_rank'])
        
        # booking_bool
        booked = np.asarray(group['booking_bool'])
        
        # click_bool
        clicked = np.asarray(group['click_bool'])
        all_scores.append(indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked))
        
    return np.nanmean(all_scores)

In [5]:
for i in range(5):
    print(i, dataframe_ndcg_score(random_solution(traindf)))

0 0.052392736120704615
1 0.04754987867397511
2 0.061032665163245695
3 0.05583035565462609
4 0.059822588022949055


In [28]:
rf = pd.read_pickle("./pickles/XGBoost_small_solution.pkl")

print(round(dataframe_ndcg_score(rf),3) * 100, "%")

3.5999999999999996 %


In [27]:
rf.head(5)

,prop_id,srch_id,price_quality,competitor_bool,availability_bools,visited_before,click_bool,booking_bool,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,importance,predicted_rank,position
1196703,2990,79994,41.3200,0,0,0,0,0,0,1,0,4.47,0.1356,0,10,24
1196704,4256,79994,22.8350,0,0,0,1,1,1,1,0,4.62,0.3492,5,3,6
1196705,6427,79994,29.3575,0,0,0,0,0,0,0,0,4.01,0.2228,0,4,17
1196706,8289,79994,34.2500,0,0,0,0,0,0,1,0,4.41,0.2138,0,21,30
1196707,13668,79994,30.0100,0,0,0,0,0,0,1,0,3.14,0.0184,0,24,21
